<a href="https://colab.research.google.com/github/isold23/RAG/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-cpp-python langchain unstructured[pdf]

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain


llm = LlamaCpp(
    model_path="/content/drive/MyDrive/chinese-llama-2-7b.Q2_K.gguf",
    input={"temperature": 1, "top_p": 1},
    n_ctx=2048,
    verbose=True,
)

from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import urllib
from time import monotonic

url = "http://www.hzdianyue.com/doc/readme.txt"

article = urllib.request.urlopen(url).read().decode("utf-8")

text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(article)

docs = [Document(page_content=t) for t in texts]

template = """
SYSTEM: Você é um especialista em resumir textos.
USER: Escreva um resumo sucinto do seguinte texto: {text}
ASSISTANT:
"""
prompt = PromptTemplate(template=template, input_variables=["text"])

chain = LLMChain(prompt=prompt, llm=llm)

chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

start_time = monotonic()
summary = chain.run(docs)
print(f"Run time {monotonic() - start_time} seconds")

print(summary)
print(len(article))
print(len(summary))